In [1]:
import os 
import sys
from datetime import date
from tqdm import tqdm
from copy import deepcopy
import numpy as np 
import pandas as pd

sys.path.append('../')
from src.datasets.data import PAMAP2Reader, DaLiAcReader, UTDReader, OPPReader

import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly_express as px 
import plotly.graph_objects as go

In [2]:
dataReader = PAMAP2Reader('../data/PAMAP2_Dataset/Protocol/')

Reading file 1 of 14
Reading file 2 of 14
Reading file 3 of 14
Reading file 4 of 14
Reading file 5 of 14
Reading file 6 of 14
Reading file 7 of 14
Reading file 8 of 14
Reading file 9 of 14
Reading file 10 of 14
Reading file 11 of 14
Reading file 12 of 14
Reading file 13 of 14
Reading file 14 of 14


g:\FYP\Codebases\HAR-ZSL-XAI\notebooks\..\src\datasets\data.py:64: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



### IMU Hand 
- 1,2,3 : (x,y,z) acceleration
- 7,8,9 : (x,y,z) gyroscope 
- 10,11,12 : (x,y,z) magnetometer

### IMU Chest
- 17,18,19 : (x,y,z) acceleration
- 23,24,25 : (x,y,z) gyroscope 
- 26,27,28 : (x,y,z) magnetometer

### IMU Ankle
- 33,34,35 : (x,y,z) acceleration
- 39,40,41 : (x,y,z) gyroscope 
- 42,43,44 : (x,y,z) magnetometer

In [3]:
cols = ['x-acc-hand', 'y-acc-hand', 'z-acc-hand', 'x-gyro-hand', 'y-gyro-hand', 'z-gyro-hand', 'x-mag-hand', 'y-mag-hand', 'z-mag-hand',
        'x-acc-chest', 'y-acc-chest', 'z-acc-chest', 'x-gyro-chest', 'y-gyro-chest', 'z-gyro-chest', 'x-mag-chest', 'y-mag-chest', 'z-mag-chest',
        'x-acc-ankle', 'y-acc-ankle', 'z-acc-ankle', 'x-gyro-ankle', 'y-gyro-ankle', 'z-gyro-ankle', 'x-mag-ankle', 'y-mag-ankle', 'z-mag-ankle',]

def show_accs(signal):
    df = pd.DataFrame(data=signal, columns=cols)
    df['time-step'] = range(df.shape[0])

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df["time-step"], y=df["x-acc-hand"], name="X-Acc-Hand", mode="lines"))
    fig.add_trace(go.Scatter(x=df["time-step"], y=df["y-acc-hand"], name="Y-Acc-Hand", mode="lines"))
    fig.add_trace(go.Scatter(x=df["time-step"], y=df["z-acc-hand"], name="Z-Acc-Hand", mode="lines"))
    fig.update_layout(
        title="3D Acceleration", xaxis_title="Time-Step", yaxis_title="Value"
    )
    fig.show()

In [25]:
def smooth(x, window_len=11, window='hanning'):
    if x.ndim != 1:
            raise Exception('smooth only accepts 1 dimension arrays.')
    if x.size < window_len:
            raise Exception("Input vector needs to be bigger than window size.")
    if window_len<3:
            return x
    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
            raise Exception("Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")
    s=np.r_[2*x[0]-x[window_len-1::-1],x,2*x[-1]-x[-1:-window_len:-1]]
    if window == 'flat': #moving average
            w=np.ones(window_len,'d')
    else:  
            w=eval('np.'+window+'(window_len)')
    y=np.convolve(w/w.sum(),s,mode='same')
    return y[window_len:-window_len+1]

In [26]:
i = 20

In [27]:
dataReader.data[i].shape, dataReader.targets[i]

((6791, 27), 11)

In [28]:
data = dataReader.data[i]
data = np.apply_along_axis(smooth, axis=0, arr=data)
data_df = pd.DataFrame(data=data, columns=cols)
data_df.head()

,x-acc-hand,y-acc-hand,z-acc-hand,x-gyro-hand,y-gyro-hand,z-gyro-hand,x-mag-hand,y-mag-hand,z-mag-hand,x-acc-chest,...,z-mag-chest,x-acc-ankle,y-acc-ankle,z-acc-ankle,x-gyro-ankle,y-gyro-ankle,z-gyro-ankle,x-mag-ankle,y-mag-ankle,z-mag-ankle
0,-10.328345,2.726463,1.199737,-0.360892,0.062213,0.180022,22.453712,-2.405343,0.486960,37.875,...,-21.829627,0.120634,35.1875,10.322420,1.786617,-0.756670,-0.250114,0.211802,-0.364279,-15.119201
1,-10.170958,2.743998,1.227032,-0.212914,0.059937,0.253011,22.686541,-2.889450,0.488462,37.875,...,-22.053411,0.120036,35.1875,10.397987,1.701016,-0.781260,-0.243357,0.236690,-0.372361,-15.095237
2,-10.013026,2.785424,1.244151,-0.072460,0.058484,0.330441,22.948981,-3.361175,0.479545,37.875,...,-22.206958,0.119330,35.1875,10.458805,1.624054,-0.820506,-0.238235,0.260326,-0.387797,-15.081597
3,-9.865935,2.851366,1.254794,0.051975,0.057046,0.411868,23.220762,-3.833890,0.445148,37.875,...,-22.273995,0.118524,35.1875,10.510441,1.560023,-0.874235,-0.234493,0.281892,-0.412501,-15.074837
4,-9.746179,2.933871,1.267914,0.148887,0.054487,0.494254,23.452385,-4.325404,0.386817,37.875,...,-22.269200,0.117700,35.1875,10.565699,1.513184,-0.933310,-0.234454,0.300478,-0.446404,-15.069035


In [29]:
data_df['time-step'] = range(data_df.shape[0])

In [30]:
data_df.shape[0]

6791

In [31]:
fig = px.line(data_df, x="time-step", y='x-acc-hand', title='3D acceleration data')
fig.show()

In [32]:
fig = px.line(data_df, x="time-step", y='x-acc-hand', title='3D acceleration data')
fig.show()

In [33]:

df = data_df
fig = go.Figure()
fig.add_trace(go.Scatter(x=df["time-step"], y=df["x-acc-hand"], name="X-Acc-Hand", mode="lines"))
fig.add_trace(go.Scatter(x=df["time-step"], y=df["y-acc-hand"], name="Y-Acc-Hand", mode="lines"))
fig.add_trace(go.Scatter(x=df["time-step"], y=df["z-acc-hand"], name="Z-Acc-Hand", mode="lines"))
fig.update_layout(
    title="3D Acceleration", xaxis_title="Date", yaxis_title="Close"
)
fig.show()

In [34]:
df = data_df
fig = go.Figure()
fig.add_trace(go.Scatter(x=df["time-step"], y=df["x-gyro-hand"], name="X-Gyro-Hand", mode="lines"))
fig.add_trace(go.Scatter(x=df["time-step"], y=df["y-gyro-hand"], name="Y-Gyro-Hand", mode="lines"))
fig.add_trace(go.Scatter(x=df["time-step"], y=df["z-gyro-hand"], name="Z-Gyro-Hand", mode="lines"))
fig.update_layout(
    title="3D Gyroscope", xaxis_title="Date", yaxis_title="Close"
)
fig.show()

In [35]:
df = data_df
fig = go.Figure()
fig.add_trace(go.Scatter(x=df["time-step"], y=df["x-mag-hand"], name="X-Mage-Hand", mode="lines"))
fig.add_trace(go.Scatter(x=df["time-step"], y=df["y-mag-hand"], name="Y-Mage-Hand", mode="lines"))
fig.add_trace(go.Scatter(x=df["time-step"], y=df["z-mag-hand"], name="Z-Mage-Hand", mode="lines"))
fig.update_layout(
    title="3D Magnetometer", xaxis_title="Date", yaxis_title="Close"
)
fig.show()

In [46]:
data_dict = dataReader.generate(unseen_classes=[1,4], seen_ratio=0.1, unseen_ratio=0.8, window_size=5.21, window_overlap=4.21, resample_freq=50)

In [47]:
data_dict['train']['X'].shape

(20531, 261, 27)

In [48]:
j = 1200
sample1 = data_dict['train']['X'][j]
show_accs(sample1)

In [39]:
j = 100
sample1 = data_dict['train']['X'][j]
show_accs(sample1)

In [39]:
j = 100
sample1 = data_dict['train']['X'][j]
show_accs(sample1)